<a href="https://colab.research.google.com/github/sungsung718/Applied-Cognitivie-Psychology/blob/main/Brain_MRI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision.datasets import ImageFolder

In [2]:
transformation = transforms.Compose([
                                     transforms.Grayscale(),
                                     transforms.Resize((32, 32)),
                                     transforms.ToTensor(),
                                     transforms.Lambda(lambda x: (x / 255.0) - 0.5)
                                     ])

dataset = ImageFolder(root = "/content/drive/MyDrive/BrainImage/brain_tumor_dataset",
                                             transform = transformation)

dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
test_size = dataset_size - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [3]:
train_loader =torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle =True) #mini batch로 나누기
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 64, shuffle = False, num_workers = 4)

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,16,kernel_size = 3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size = 3, padding=1)
        self.fc1 = nn.Linear(8*8*8, 32)
        self.fc2 = nn.Linear(32,10)
        self.dropout = nn.Dropout(0.2)
        self.batchnormalize = nn.BatchNorm2d(16)
    def forward(self,x):
        out = self.conv1(x)
        out = self.batchnormalize(out)
        out = F.max_pool2d(torch.tanh(out), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8*8*8)
        out = torch.tanh(self.fc1(out))
        out = self.dropout(out)
        out = self.fc2(out)
        return out

model = Net().cuda()
LR= 0.01
optimizer = optim.SGD(model.parameters(), lr=LR)
loss_fn = nn.CrossEntropyLoss()
n_epochs = 20

for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        imgs, labels = imgs.cuda(), labels.cuda()
        outputs = model(imgs)
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad() #gradient을 0으로 초기화
        loss.backward() #역전파
        optimizer.step() #parameter update
    print(f'Epoch: {epoch}, Loss: {float(loss)}')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: 0, Loss: 2.2466158866882324
Epoch: 1, Loss: 2.012960910797119
Epoch: 2, Loss: 1.7423089742660522
Epoch: 3, Loss: 1.4400606155395508
Epoch: 4, Loss: 1.3816896677017212
Epoch: 5, Loss: 1.2244049310684204
Epoch: 6, Loss: 1.0217609405517578
Epoch: 7, Loss: 1.3096354007720947
Epoch: 8, Loss: 1.0151116847991943
Epoch: 9, Loss: 0.8440475463867188
Epoch: 10, Loss: 1.0583118200302124
Epoch: 11, Loss: 0.7847738265991211
Epoch: 12, Loss: 0.8970576524734497
Epoch: 13, Loss: 0.8276912569999695
Epoch: 14, Loss: 0.7303128242492676
Epoch: 15, Loss: 0.7203853130340576
Epoch: 16, Loss: 0.8269745111465454
Epoch: 17, Loss: 0.8832510709762573
Epoch: 18, Loss: 0.6974087953567505
Epoch: 19, Loss: 0.676308274269104


In [4]:
correct = 0
total = 0
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.cuda(), labels.cuda()
        outputs = model(imgs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct // total} %')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Accuracy: 60 %
